In [1]:
from functools import reduce
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col, coalesce, split, regexp_replace, when, length, get_json_object, explode, size, array_contains

In [2]:
spark = SparkSession.builder \
    .appName("medi_test") \
    .getOrCreate()

23/12/05 16:58:26 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
root_path = '/Users/b06/Desktop/yeardream/medi-05'
json_root_path = f'{root_path}/data/naverplace_meta'
save_root_path = f'{root_path}/spark-scala-project/output/pyspark'
text_root_path = f'{root_path}/spark-scala-project/test.txt'

In [4]:
test_json_path = f'{json_root_path}/naverplace_meta_1.json'

In [5]:
df = spark.read.option("multiline", "true").json(test_json_path)
# df = spark.read.option("multiline", "true").json('/Users/b06/Desktop/yeardream/medi-05/data/test.json')

23/12/05 16:58:27 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
hospital_df = df.select(explode("Hospital").alias("hospital"))
root_query_df = df.select(explode("ROOT_QUERY").alias("root_query"))

In [7]:
# hospital_df.select("hospital.id").show()
# hospital_df.select("hospital.name").filter(hospital_df.hospital.id == '11779766').show()

In [8]:
# hospital_df.show()
# hospital_df.printSchema(2)

In [9]:
# visitorReviewsTotal
# visitorReviewsScore
# visitorReviewsTextReviewTotal
# photo_review_ratio = (visitorReviewsTotal-visitorReviewsTextReviewTotal)/visitorReviewsTotal
# images
# images_count len(images)
# is_smart_phone bool: if phone.startswith('010')
# is_zeropay bool: if '제로페이' in payment_info
homepages_url cond1: [homepages.repr.url] and cond2: [homepages.repr.etc.url]
homepages_type cond1: [homepages.repr.type] and cond2: [homepages.repr.etc.type]
homepages_order: len(homepages.etc.order) + 1
homepages_isDeadurl cond1: [homepages.repr.isDeadUrl] and cond2: [homepages.repr.isDeadUrl]
homepages_isRep homepages.repr.url
description_length len(description)
is_blog_exposed ?
self_blog_present ?

SyntaxError: invalid syntax (3955356764.py, line 9)

In [10]:
hospital_df = hospital_df.select(
    col("hospital.id").alias("id"),
    col("hospital.name").alias("name"),
    col("hospital.reviewSettings.keyword").alias("review_keyword"),
    col("hospital.keywords").alias("keywords"),
    col("hospital.naverBookingUrl").alias("booking_url"),
    col("hospital.talktalkUrl").alias("talktalk_url"),
    col("hospital.category").alias("category"),
    col("hospital.categoryCode").alias("category_code"),
    col("hospital.categoryCodeList").alias("category_code_list"),
    col("hospital.categoryCount").alias("category_count"),
    col("hospital.conveniences").alias("conveniences"),
    col("hospital.paymentInfo").alias("payment_info"),
    col("hospital.rcode").alias("rcode"),
    col("hospital.virtualPhone").alias("virtual_phone"),
    col("hospital.phone").alias("phone"),
    col("hospital.roadAddress").alias("road_address"),
    col("hospital.road").alias("road"),
    col("hospital.description").alias("description"),
    col("hospital.bookingBusinessId").alias("booking_business_id"),
    col("hospital.bookingDisplayName").alias("booking_display_name"),
    col("hospital.visitorReviewsScore").alias("visitor_reviews_score"),
    col("hospital.visitorReviewsTotal").alias("visitor_reviews_total"),
    col("hospital.visitorReviewsTextReviewTotal").alias("visitor_reviews_text_review_total"),
    col("hospital.images").alias("images"),
    col("hospital.homepages.etc").alias("homepages_etc"),
    col("hospital.homepages.repr").alias("homepages_repr"),
)

In [11]:
hospital_df = hospital_df.withColumn(
    "images_count", 
    size("images")
)
# hospital_df.select("images_count").show()

In [12]:
hospital_df = hospital_df.withColumn(
    'photo_review_ratio',
    (col('visitor_reviews_total')-col('visitor_reviews_text_review_total'))/col('visitor_reviews_total')
)
# hospital_df.select("photo_review_ratio").show()

In [13]:
hospital_df = hospital_df.withColumn(
    'is_smart_phone',
    col('phone').startswith('010')
)
# hospital_df.select("is_smart_phone").show()

In [14]:
hospital_df = hospital_df.withColumn(
    'is_zero_pay',
    array_contains(col('payment_info'), '제로페이')
)
# hospital_df.select("is_zero_pay").show()

In [19]:
# hospital_df.select("homepages_etc.url").collect()[1]

Row(url=['https://blog.naver.com/kundaeclinic', 'https://www.instagram.com/bluehillclinic_official'])